In [12]:
import pandas as pd
#import numpy as np

from bokeh.models import ColumnDataSource, Column
from bokeh.models.tools import HoverTool
from bokeh.plotting import figure
from bokeh.io import show, output_file, output_notebook
from bokeh.models.widgets import CheckboxGroup
from bokeh.layouts import row, WidgetBox
from bokeh.application.handlers import FunctionHandler
from bokeh.application import Application

import datetime as dt
import random

df = pd.read_csv("https://epistat.sciensano.be/Data/COVID19BE_CASES_AGESEX.csv", encoding="ISO-8859-1", index_col="DATE", parse_dates=True)
df_old = pd.read_csv("https://epistat.sciensano.be/Data/20200403/COVID19BE_CASES_AGESEX_20200403.csv", encoding="ISO-8859-1", index_col="DATE", parse_dates=True)

muni = pd.read_csv("https://epistat.sciensano.be/Data/COVID19BE_CASES_MUNI.csv", encoding="ISO-8859-1", index_col="DATE", parse_dates=True)
muni_old = pd.read_csv("https://epistat.sciensano.be/Data/20200403/COVID19BE_CASES_MUNI_20200403.csv", encoding="ISO-8859-1", index_col="DATE", parse_dates=True)

muni_cum = pd.read_csv("https://epistat.sciensano.be/Data/COVID19BE_CASES_MUNI_CUM.csv", encoding="ISO-8859-1")
muni_cum_old = pd.read_csv("https://epistat.sciensano.be/Data/20200403/COVID19BE_CASES_MUNI_CUM_20200403.csv", encoding="ISO-8859-1")

hosp = pd.read_csv("https://epistat.sciensano.be/Data/COVID19BE_HOSP.csv", encoding="ISO-8859-1", index_col="DATE", parse_dates=True)
hosp_old = pd.read_csv("https://epistat.sciensano.be/Data/20200403/COVID19BE_HOSP_20200403.csv", encoding="ISO-8859-1", index_col="DATE", parse_dates=True)

deaths = pd.read_csv("https://epistat.sciensano.be/Data/COVID19BE_MORT.csv", encoding="ISO-8859-1", index_col="DATE", parse_dates=True)
deaths_old = pd.read_csv("https://epistat.sciensano.be/Data/20200403/COVID19BE_MORT_20200403.csv", encoding="ISO-8859-1", index_col="DATE", parse_dates=True)

tests = pd.read_csv("https://epistat.sciensano.be/Data/COVID19BE_tests.csv", encoding="ISO-8859-1", index_col="DATE", parse_dates=True)
tests_old = pd.read_csv("https://epistat.sciensano.be/Data/20200403/COVID19BE_tests_20200403.csv", encoding="ISO-8859-1", index_col="DATE", parse_dates=True)


In [54]:
french = pd.read_csv("https://raw.githubusercontent.com/opencovid19-fr/data/master/data-sources/sante-publique-france/covid_hospit.csv",\
                     sep=";", index_col="jour", parse_dates=True)

,hosp,rea,rad,dc
jour,,,,
2020-03-18,5905,1535,1627,435
2020-03-19,7961,1975,2322,642
2020-03-20,10272,2564,3128,890
2020-03-21,11599,2872,3580,1041
2020-03-22,13675,3311,4188,1251
2020-03-23,17087,4114,5084,1707
2020-03-24,20181,4996,6514,2186
2020-03-25,23992,5847,8113,2763
2020-03-26,27601,6676,9827,3379


In [57]:
#french = french.groupby('jour').sum().drop('sexe', axis=1)
list_cat = ['hosp', 'rea', 'dc']
french_bar = french[list_cat].reset_index().melt(['jour']).set_index('jour').sort_index() #adios tidy data :/
french_bar

,variable,value
jour,,
2020-03-18,hosp,5905
2020-03-18,rea,1535
2020-03-18,dc,435
2020-03-19,hosp,7961
2020-03-19,rea,1975
...,...,...
2020-04-06,hosp,58813
2020-04-06,rea,13832
2020-04-07,hosp,59430


In [45]:
df.groupby('DATE').sum().merge(df_old.groupby('DATE').sum(), how='right')


,CASES
0,16
1,16
2,17
3,36
4,54
5,81
6,107
7,27
8,64
9,94


In [21]:
tmp = pd.DataFrame(df.groupby('DATE').sum()['CASES'])

df2 = deaths.groupby("DATE").sum().merge(hosp.groupby("DATE").sum(), on="DATE")
df2 = tmp.merge(df2, on="DATE", how='left').fillna(0).astype(int)
categories = list(df2.columns)

In [26]:
df2

,CASES,DEATHS,NR_REPORTING,TOTAL_IN,TOTAL_IN_ICU,TOTAL_IN_RESP,TOTAL_IN_ECMO,NEW_IN,NEW_OUT
DATE,,,,,,,,,
2020-03-01,16,0,0,0,0,0,0,0,0
2020-03-02,17,0,0,0,0,0,0,0,0
2020-03-03,36,0,0,0,0,0,0,0,0
2020-03-04,54,0,0,0,0,0,0,0,0
2020-03-05,81,0,0,0,0,0,0,0,0
2020-03-06,107,0,0,0,0,0,0,0,0
2020-03-07,27,0,0,0,0,0,0,0,0
2020-03-08,64,0,0,0,0,0,0,0,0
2020-03-09,94,0,0,0,0,0,0,0,0


In [5]:
def make_dataset(list_cat):
    df = df2[list_cat]
    plot_df = df.reset_index().melt(['DATE']).set_index('DATE').sort_index() #adios tidy data :/
    col_gen = lambda : "#"+''.join([random.choice('0123456789ABCDEF') for j in range(6)])
    colors = {i:col_gen() for i in categories}
    plot_df["color"] = plot_df["variable"].map(colors)

    return plot_df

In [6]:
def make_plot(src):
    # create a new plot with a datetime axis type
    p = figure(plot_width=700, plot_height=700, x_axis_type="datetime")

    p.vbar(x='DATE', top='value', source = src, fill_alpha = 0.7,\
           hover_fill_alpha = 1.0, line_color = 'black', width=dt.timedelta(1), color='color')

    #define tooltips    
    p.add_tools(HoverTool(tooltips = [('Date', '@DATE{%F}'), ('', '@variable: @value')], formatters={'@DATE': 'datetime'}, point_policy="follow_mouse"))

    # attributes
    p.title.text = "Count"
    p.legend.location = "top_left"
    p.grid.grid_line_alpha = 0
    p.xaxis.axis_label = 'Date'
    p.yaxis.axis_label = 'Daily new cases'
    p.ygrid.band_fill_color = "olive"
    p.ygrid.band_fill_alpha = 0.1

    return p

In [7]:
def update(attr, old, new):
    cat_to_plot = [cat_selection.labels[i] for i in cat_selection.active]
    new_df = make_dataset(cat_to_plot)
    new_df=ColumnDataSource(new_df)
    src.data.update(new_df.data)

In [8]:
def modify_doc(doc):
    cat_selection = CheckboxGroup(labels=categories, active=[0, 1, 2])
    cat_selection.on_change('active', update)
    
    controls = Column(cat_selection)
    init_cat = [cat_selection.labels[i] for i in cat_selection.active]
    
    src = make_dataset(init_cat) 
    p = make_plot(src)
    layout = row(controls, p)
    doc.add_root(layout)

In [9]:
handler = FunctionHandler(modify_doc)
app = Application(handler)
output_notebook()
show(app)

Loading BokehJS ...

/home/simon/anaconda3/envs/daily/lib/python3.7/site-packages/bokeh/models/plots.py:764: UserWarning: 
You are attempting to set `plot.legend.location` on a plot that has zero legends added, this will have no effect.

Before legend properties can be set, you must add a Legend explicitly, or call a glyph method with a legend parameter set.

  warnings.warn(_LEGEND_EMPTY_WARNING % attr)
ERROR:bokeh.server.views.ws:Refusing websocket connection from Origin 'http://ipad-pro-de-j:8888';                       use --allow-websocket-origin=ipad-pro-de-j:8888 or set BOKEH_ALLOW_WS_ORIGIN=ipad-pro-de-j:8888 to permit this; currently we allow origins {'localhost:8888'}


In [53]:
df2[['CASES', 'DEATHS', 'TOTAL_IN', 'TOTAL_IN_ICU', 'NEW_IN', 'NEW_OUT']]
line_cat = ['NEW_IN', 'NEW_OUT']
categories = list(df2.columns)
df_line = df2[[i for i in categories if i in line_cat]]

plot_df_line = df_line.reset_index().melt(['DATE']).set_index('DATE').sort_index() #adios tidy data :/


src_line =ColumnDataSource(df_line)

colors = {'NEW_IN':'red', 'NEW_OUT':'green'}
p = figure(plot_width=700, plot_height=700, x_axis_type="datetime")

for col in src_line.data:
    if col != 'DATE':
        p.line(x='DATE', y=col, source=src_line, line_width=4, color=colors[col])
show(p)